In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
gender = pd.read_csv('gender_submission.csv')

In [3]:
def surviving_score(train):
    # Define influencing parameters
    influence_params = ['Sex', 'SibSp', 'Parch', 'Fare Group', 'Age Group','Pclass']
    
    train['Age'] =  train['Age'].fillna(value = 101 )
    
    # Create age groups for 'Age'
    train['Age Group'] = pd.cut(train['Age'], bins=list(range(0, 103)), labels=False, right=False)
    
    # Create fare groups for 'Fare'
    train['Fare Group'] = pd.cut(train['Fare'], bins=list(range(0, 5000, 2)), labels=False, right=False)

    # Dictionary to store survival rates for each category in each parameter
    survival_rates = {}

    for param in influence_params:
        survival_rates[param] = train.groupby(param)['Survived'].mean()

    survival_rates_df = pd.DataFrame([(key, var, val) for key, series in survival_rates.items() for var, val in series.items()], columns=['Parameter', 'Category', 'Survival Rate'])

    return survival_rates_df



In [4]:
def final_calcul(train, survival_rates_df):
    influence_params = ['Sex', 'SibSp', 'Parch', 'Fare Group', 'Age Group','Pclass']
    
    # Create age and fare groups if not already present
    train['Age'] =  train['Age'].fillna(value = 101 )
    train['Age Group'] = pd.cut(train['Age'], bins=list(range(0, 103)), labels=False, right=False)
    train['Fare Group'] = pd.cut(train['Fare'], bins=list(range(0, 5000, 10)), labels=False, right=False)

    # Calculate composite survival score for each passenger
    for index, row in train.iterrows():
        scores = []
        for param in influence_params:
            # Find the survival rate for the passenger's category in each parameter
            category = row[param]
            survival_rate = survival_rates_df[(survival_rates_df['Parameter'] == param) & (survival_rates_df['Category'] == category)]['Survival Rate']
            
            if not survival_rate.empty:
                scores.append(survival_rate.iloc[0])
        
        # Calculate the average score (you might choose a different method)
        avg_score = sum(scores) / len(scores) if scores else 0
        train.at[index, 'Survival Score'] = avg_score

    # Determine survival prediction (you need to decide a threshold)
    threshold = 0.40  # Example threshold
    train['Survived prediction'] = train['Survival Score'].apply(lambda x: 1 if x >= threshold else 0)
    
    return train

In [5]:
surviving_score = surviving_score(train)
surviving_score

,Parameter,Category,Survival Rate
0,Sex,female,0.742038
1,Sex,male,0.188908
2,SibSp,0,0.345395
3,SibSp,1,0.535885
4,SibSp,2,0.464286
...,...,...,...
147,Age Group,80,1.000000
148,Age Group,101,0.293785
149,Pclass,1,0.629630
150,Pclass,2,0.472826


In [6]:
final_calcul = final_calcul(train, x)

NameError: name 'x' is not defined

In [ ]:
final_calcul

In [ ]:
output = y[['PassengerId','Survived']]
output

In [ ]:
output.to_csv('train_02.csv',index=False)